Following Free Code Camps tutorial closely.

In [1]:
import ollama
import numpy as np
import torch

In [2]:
from langchain_ollama import OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings

model_name="sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(model = model_name,
                                   encode_kwargs={'normalize_embeddings': True})

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Lecture 1 Magnetic fields and magnetic force.pdf")
loader2 = PyPDFLoader("Lecture 4 Faraday’s law and induction.pdf")
docs = loader.load()
docs2 = loader2.load()

print(f"Total characters: {len(docs[2].page_content)}")
print(f"Total characters: {len(docs2[5].page_content)}")

Total characters: 1647
Total characters: 1765


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_docs = docs + docs2

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 512,
    chunk_overlap = 64
)

#Make splits
splits = text_splitter.split_documents(all_docs)

In [5]:
len(all_docs)

17

In [6]:
print(docs[0].page_content[:500])
print(docs2[0].page_content[:500])

PHYS2003: Physics for Electrical Engineers, semester 1 2023 
Lecture 1 
1 
 
Lecture 1: Magnetic fields and magnetic force 
Learning outcomes 
Assessable 
• Calculate the force on moving charges in a magnetic field via the cross product. 
• Calculate the direction of resulting vectors from the cross product or, where 
appropriate, apply basic rules to determine the direction. 
• Calculate potential difference in a moving conductor. 
Understanding 
• Visualize and conceptually explain the motion 
PHYS2003: Physics for Electrical Engineers, semester 1 2023 
Lecture 4 
1 
 
Lecture 4: Faraday’s law and induction 
Learning outcomes 
Assessable 
• Use Faraday’s law and Lenz’s law to calculate induced current and emf. 
• Calculate energy transfer due to induction. 
Understanding 
• Visualize and conceptually explain induced currents and electric fields. 
1. Summary 
This lecture will cover Faraday’s law, Lenz’s law, and induction and energy transfer. 
Consider a loop of conducting wire conne

In [7]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,  # these are already LangChain `Document` objects
    embedding=embeddings,
    collection_name="circuit_docs",
    persist_directory="./chroma_db"
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

From RAG Tutorial this appears to end the embedding and indexing section, next video we will see retrieval

# Part3 - Retrieval

In this section code camp goes over the retrieval section

In [8]:
docsearch = retriever.invoke("Edwin H. Hall experiment")

In [9]:
docsearch

[Document(id='1a0fde3d-57bd-4055-95e4-1f9775d808a8', metadata={'creationdate': "D:20230224061506Z00'00'", 'source': 'Lecture 1 Magnetic fields and magnetic force.pdf', 'moddate': "D:20230305080614Z00'00'", 'total_pages': 7, 'creator': 'Microsoft® Word 2016', 'page': 5, 'page_label': '6', 'producer': 'macOS Version 12.5 (Build 21G72) Quartz PDFContext, AppendMode 1.1', 'author': 'David Gozzard'}, page_content='PHYS2003: Physics for Electrical Engineers, semester 1 2023 \nLecture 1 \n6 \n \n \nFigure 3: Simplified diagram of the J. J. Thomson experiment that discovered the electron. \n5.2. The Hall Effect \nIn a conductor, when there is no current through it, the electrons move randomly with no \nnet or average motion in any direction. When a current is passed through the conductor, \nthe electrons continue to move randomly, but with an average drift in the opposite \ndirection to the electric field causing the current. The drift speed 𝑣𝑑 is very smaller \ncompared with the speed of the 

In [10]:
docsearch2 = retriever.invoke("Explain Faraday's and Lenz's law")

In [11]:
print(docsearch2)

[Document(id='ef742050-ae4a-4dc7-a830-bb76a20e31a1', metadata={'source': 'Lecture 4 Faraday’s law and induction.pdf', 'page': 0, 'creator': 'Microsoft® Word 2016', 'creationdate': "D:20230224061645Z00'00'", 'author': 'David Gozzard', 'producer': 'macOS Version 12.5 (Build 21G72) Quartz PDFContext, AppendMode 1.1', 'page_label': '1', 'total_pages': 10, 'moddate': "D:20230311062840Z00'00'"}, page_content='PHYS2003: Physics for Electrical Engineers, semester 1 2023 \nLecture 4 \n1 \n \nLecture 4: Faraday’s law and induction \nLearning outcomes \nAssessable \n• Use Faraday’s law and Lenz’s law to calculate induced current and emf. \n• Calculate energy transfer due to induction. \nUnderstanding \n• Visualize and conceptually explain induced currents and electric fields. \n1. Summary \nThis lecture will cover Faraday’s law, Lenz’s law, and induction and energy transfer. \nConsider a loop of conducting wire connected to an ammeter. If we move a bar magnet \ntowards the loop, we will observe: 

# Part 4 - Generation

In [12]:
from langchain.prompts import PromptTemplate

template = """Answer the following question only using the following context:
{context}

If the answer is not contained in the context, respond with:
"I cannot answer this question because the necessary information was not found in the provided documents."

When answering, include the **source file name** and **slide/page number** if available.

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the following question only using the following context:\n{context}\n\nIf the answer is not contained in the context, respond with:\n"I cannot answer this question because the necessary information was not found in the provided documents."\n\nWhen answering, include the **source file name** and **slide/page number** if available.\n\nQuestion: {question}\n')

In [13]:
llm = OllamaLLM(model = "llama3.2")

In [14]:
chain = prompt | llm

In [15]:
print(chain.invoke({"context":docsearch, "question":"Describe an experiment used to find the Hall effect"}))

The Hall Effect is described as an experiment where a current is passed through a conductor, causing electrons to drift randomly in one direction. When a magnetic field is applied perpendicular to the conductor, it deflects the drifting electrons, creating a separation of charges and resulting in an electric field within the conductor.

Source file name: Lecture 1 Magnetic fields and magnetic force.pdf
Slide/page number: Page 5 (document '5edf78b0-503c-41ba-993a-752cb731f07d')


In [16]:
print(chain.invoke({"context":docsearch2, "question":"Explain Faraday's Law and how it relates to induction"}))

The source file is 'Lecture 4 Faraday’s law and induction.pdf' (page 1).

According to the text, Faraday's Law of Induction states that "Induced emf and current is caused by changing the amount of magnetic field passing through the conducting loop." This is defined as magnetic flux (𝛷𝐵), which is calculated as: 𝛷𝐵 = ∫𝐵⃗ ∙ 𝑑𝐴, where 𝐵⃗  is the magnetic field enclosed by a conducting loop of area 𝐴.

In the case where the magnetic field is uniform, we can write the dot product as 𝐵 𝑑𝐴 cos0 = 𝐵 𝑑𝐴. This allows us to bring out the front of the integral and get: 𝛷𝐵 = 𝐵𝐴.

The unit of magnetic flux is tesla-square meters (weber), which is defined as 1 weber = 1 T ⋅ m².

Formally, Faraday's Law can be written as: ℰ = − ∫(𝑑𝛷𝐵)/𝑑𝑡, where the minus sign indicates that the induced emf acts to oppose the change of flux in the loop.

In an ideal (closely packed) coil of N turns, the equation becomes: ℰ = −𝑁 ∫(𝑑𝛷𝐵)/𝑑𝑡.
